This is a notebook to recreate Figure 6 of [Lamb, Taylor & van Haasteren 2023](https://arxiv.org/abs/2303.15442). This notebook analyses the evolution of spectral parameterisation with number of pulsars and frequency

In [1]:
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format ='retina'

import numpy as np
import matplotlib.pyplot as plt
import glob
from chainconsumer import ChainConsumer
import pickle
from ceffyl import Ceffyl, models
from enterprise.signals import parameter, gp_priors as gpp
from enterprise_extensions.model_utils import get_tspan
from la_forge.slices import SlicesCore
import json
import la_forge.core as co
from tqdm import tqdm

# ACCRE-specific import to load correct latex file
## COMMENT OUT AS REQUIRED
import os
os.environ["PATH"] += os.pathsep + '/home/lambwg/latex/bin/x86_64-linux' 

In [2]:
# setup default plotting code
plt.rcParams.update(plt.rcParamsDefault)
with open('/home/lambwg/ng15_rcparams.json', 'rb') as fin:
    plt.rcParams.update(json.load(fin))
    
plt.rcParams["figure.figsize"] = [3.5503666805036667*2, 2.1942472810764047*2]

In [3]:
# load injected values
inj = np.loadtxt('../../data/sim51/injected_values.txt')

# load all the chains!

## credible regions as a function of $N_\mathrm{f}$

I ran some refits for increasing numbers of frequencies. This is trivial with ceffyl and left as an exercise :)

##### free spectrum refit

In [29]:
# load paths to files of chains from FREE SPECTRUM REFIT
chains = [f'/data/taylor_group/william_lamb/GFL/realisation_51/fs2A/fftkde_10k_2/refits/m2aperfreq/{ii}f/'
          for ii in range(1,11)]

params = [f'{ii}f' for ii in range(1,11)]  # label chains

# use la_forge's SlicesCore class to only load log10_A and gamma
log10Afs = SlicesCore(slicedirs=chains, pars2pull='log10_A_gw',
                     params=params)
gammafs = SlicesCore(slicedirs=chains, pars2pull='gamma_gw',
                     params=params)

# calculate credible regions of the posterior
q2p5Afs, q16Afs, q50Afs, q84Afs, q97p5Afs = np.percentile(log10Afs(params),
                                                          [2.5, 16, 50, 84, 97.5],
                                                          axis=0)
q2p5gfs, q16gfs, q50gfs, q84gfs, q97p5gfs = np.percentile(gammafs(params),
                                                          [2.5, 16, 50, 84, 97.5],
                                                          axis=0)

/data/taylor_group/william_lamb/GFL/realisation_51/fs2A/fftkde_10k_2/refits/m2aperfreq/10f//chain_1.txt is loaded.

/data/taylor_group/william_lamb/GFL/realisation_51/fs2A/fftkde_10k_2/refits/m2aperfreq/10f//chain_1.txt is loaded.



In [30]:
# load paths to files of chains from FREE SPECTRUM REFIT
chains = [f'/data/taylor_group/william_lamb/GFL/realisation_51/fs2A/fftkde_10k_2/refits/m2aperfreq_down/{ii}f/'
          for ii in range(1,11)]

params = [f'{ii}f' for ii in range(1,11)]  # label chains

# use la_forge's SlicesCore class to only load log10_A and gamma
log10Afsd = SlicesCore(slicedirs=chains, pars2pull='log10_A_gw',
                      params=params)
gammafsd = SlicesCore(slicedirs=chains, pars2pull='gamma_gw',
                      params=params)

# calculate credible regions of the posterior
q2p5Afsd, q16Afsd, q50Afsd, q84Afsd, q97p5Afsd = np.percentile(log10Afsd(params),
                                                               [2.5, 16, 50, 84, 97.5],
                                                               axis=0)
q2p5gfsd, q16gfsd, q50gfsd, q84gfsd, q97p5gfsd = np.percentile(gammafsd(params),
                                                               [2.5, 16, 50, 84, 97.5],
                                                               axis=0)

/data/taylor_group/william_lamb/GFL/realisation_51/fs2A/fftkde_10k_2/refits/m2aperfreq_down/10f//chain_1.txt is loaded.

/data/taylor_group/william_lamb/GFL/realisation_51/fs2A/fftkde_10k_2/refits/m2aperfreq_down/10f//chain_1.txt is loaded.



##### GFL Lite

In [26]:
# load paths to files of chains from GFL LITE
chains = [f'/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aperfreq_reflect/{ii}f/'
          for ii in range(1,11)]

params = [f'{ii}f' for ii in range(1,11)]  # create labels for chains

q2p5A, q16A, q50A, q84A, q97p5A = [], [], [], [], []
q2p5g, q16g, q50g, q84g, q97p5g = [], [], [], [], []

for ii in range(10):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aperfreq_reflect/{ii+1}f/chain.core')
    
    q2p5A.append(np.percentile(c0('gwlog10_A'), 2.5))
    q16A.append(np.percentile(c0('gwlog10_A'), 16))
    q50A.append(np.percentile(c0('gwlog10_A'), 50))
    q84A.append(np.percentile(c0('gwlog10_A'), 84))
    q97p5A.append(np.percentile(c0('gwlog10_A'), 97.5))
    
    q2p5g.append(np.percentile(c0('gwgamma'), 2.5))
    q16g.append(np.percentile(c0('gwgamma'), 16))
    q50g.append(np.percentile(c0('gwgamma'), 50))
    q84g.append(np.percentile(c0('gwgamma'), 84))
    q97p5g.append(np.percentile(c0('gwgamma'), 97.5))

Same but for decreasing numbers of frequencies, from highest to lowest!

In [28]:
# load paths to files of chains from GFL LITE
chains = [f'/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aperfreq_down_reflect/{ii}f//'
          for ii in range(1,11)]

params = [f'{ii}f' for ii in range(1,11)]  # labels

# load paths to files of chains from GFL LITE
chains = [f'/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aperfreq_down_reflect/{ii}f//'
          for ii in range(1,11)]

params = [f'{ii}f' for ii in range(1,11)]  # labels

q2p5Ad, q16Ad, q50Ad, q84Ad, q97p5Ad = [], [], [], [], []
q2p5gd, q16gd, q50gd, q84gd, q97p5gd = [], [], [], [], []

# calculate credible regions of the posterior
for ii in range(10):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aperfreq_down_reflect/{ii+1}f/chain.core')
    
    q2p5Ad.append(np.percentile(c0('gwlog10_A'), 2.5))
    q16Ad.append(np.percentile(c0('gwlog10_A'), 16))
    q50Ad.append(np.percentile(c0('gwlog10_A'), 50))
    q84Ad.append(np.percentile(c0('gwlog10_A'), 84))
    q97p5Ad.append(np.percentile(c0('gwlog10_A'), 97.5))
    
    q2p5gd.append(np.percentile(c0('gwgamma'), 2.5))
    q16gd.append(np.percentile(c0('gwgamma'), 16))
    q50gd.append(np.percentile(c0('gwgamma'), 50))
    q84gd.append(np.percentile(c0('gwgamma'), 84))
    q97p5gd.append(np.percentile(c0('gwgamma'), 97.5))

# Plot the figure!

In [31]:
freqs = np.load('/data/taylor_group/william_lamb/GFL/realisation_51/spsrs_10fCP_10firn/fftkde_10000_final/freqs.npy')

In [32]:
(1/(15*365.24*24*60*60))*10**9

2.112597027356272

In [33]:
freqs*10**9

array([ 2.09568132,  4.19136264,  6.28704396,  8.38272528, 10.4784066 ,
       12.57408792, 14.66976924, 16.76545057, 18.86113189, 20.95681321])

In [34]:
fig, ((ax1, ax2),
      (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, tight_layout=True,
                                 sharex=True, sharey='row')

f = np.arange(1, 11)

ax1.set_title('Free spectrum refit')
ax2.set_title('GFL Lite')

ax1.plot(f, q50Afs, c='C0', label='Increasing frequency')
ax1.fill_between(f, q16Afs, q84Afs, alpha=0.4, color='C0')
ax1.fill_between(f, q2p5Afs, q97p5Afs, alpha=0.2, color='C0')
ax1.axhline(inj[0], c='k', ls='--');

ax3.plot(f, q50gfs, c='C0')
ax3.fill_between(f, q16gfs, q84gfs, alpha=0.4, color='C0')
ax3.fill_between(f, q2p5gfs, q97p5gfs, alpha=0.2, color='C0')
ax3.axhline(inj[1], c='k', ls='--')

ax1.plot(f, q50Afsd, c='grey', label='Decreasing frequency', alpha=0.75)
ax1.fill_between(f, q16Afsd, q84Afsd, alpha=0.1, color='grey')
ax1.fill_between(f, q2p5Afsd, q97p5Afsd, alpha=0.05, color='grey')
ax1.axhline(inj[0], c='k', ls='--');

ax3.plot(f, q50gfsd, c='grey', alpha=0.75)
ax3.fill_between(f, q16gfsd, q84gfsd, alpha=0.1, color='grey')
ax3.fill_between(f, q2p5gfsd, q97p5gfsd, alpha=0.05, color='grey')
ax3.axhline(inj[1], c='k', ls='--')

ax1.set_xlim(1,10)

ax2.plot(f, q50A, color='C0', label='Increasing frequency')
ax2.fill_between(f, q16A, q84A, alpha=0.4, color='C0')
ax2.fill_between(f, q2p5A, q97p5A, alpha=0.2, color='C0')
ax2.axhline(inj[0], c='k', ls='--');

ax4.plot(f, q50g, color='C0')
ax4.fill_between(f, q16g, q84g, alpha=0.4, color='C0')
ax4.fill_between(f, q2p5g, q97p5g, alpha=0.2, color='C0')
ax4.axhline(inj[1], c='k', ls='--')

ax2.plot(f, q50Ad, color='grey', alpha=0.75, label='Decreasing frequency')
ax2.fill_between(f, q16Ad, q84Ad, alpha=0.1, color='grey')
ax2.fill_between(f, q2p5Ad, q97p5Ad, alpha=0.05, color='grey')
ax2.axhline(inj[0], c='k', ls='--');

ax4.plot(f, q50gd, color='grey', alpha=0.75)
ax4.fill_between(f, q16gd, q84gd, alpha=0.1, color='grey')
ax4.fill_between(f, q2p5gd, q97p5gd, alpha=0.05, color='grey')
ax4.axhline(inj[1], c='k', ls='--')

ax3.set_xlabel('Number of frequencies')
ax4.set_xlabel('Number of frequencies')

ax3.set_xticks(np.arange(1, 11))
ax4.set_xticks(np.arange(1, 11))

ax1.set_ylabel('$\log_{10}A$')
ax3.set_ylabel('spectral index $\gamma$')

#ax4.set_xticks(ticks=np.arange(1,11),
#               labels=[f'{ii}/T' for ii in range(1,11)])

secx = ax1.secondary_xaxis('top')
secx.set_xticks(np.arange(1, 11))
labels1 = secx.set_xticklabels([f'{f:.1f}' for f in freqs*10**9])
secx.set_xlabel('GW frequencies [nHz]')

secx = ax2.secondary_xaxis('top')
secx.set_xticks(np.arange(1, 11))
labels2 = secx.set_xticklabels([f'{f:.1f}' for f in freqs*10**9])
secx.set_xlabel('GW frequencies [nHz]')

[lab.set_color('C0') for lab in labels1]
[lab.set_color('C0') for lab in labels2]

ax3.set_xlim(1,10)
ax2.legend(markerfirst=False);

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_rasterized(True)

plt.show();
fig.savefig('../plots/per_freq.pdf',
            dpi=400, bbox_inches='tight');

# credible regions as a function of $N_p$

Same as above, but increasing number of pulsars. Number of pulsars increased by adding longest or shortest timed pulsars

## load the data!

In [36]:
datadir = '/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/'

downA = SlicesCore(slicedirs=[datadir+f'refits/m2aNpsr_down/{ii}psr/' for ii in range(1, 46)],
                   params=[f'{ii}psr' for ii in range(1, 46)], pars2pull=['gwlog10_A'])
downg = SlicesCore(slicedirs=[datadir+f'refits/m2aNpsr_down/{ii}psr/' for ii in range(1, 46)],
                   params=[f'{ii}psr' for ii in range(1, 46)], pars2pull=['gwgamma'])
upA = SlicesCore(slicedirs=[datadir+f'refits/m2aNpsr_up/{ii}psr/' for ii in range(1, 46)],
                 params=[f'{ii}psr' for ii in range(1, 46)], pars2pull=['gwlog10_A'])
upg = SlicesCore(slicedirs=[datadir+f'refits/m2aNpsr_up/{ii}psr/' for ii in range(1, 46)],
                 params=[f'{ii}psr' for ii in range(1, 46)], pars2pull=['gwgamma'])

params=[f'{ii}psr' for ii in range(1, 46)]

q2p5Aup, q16Aup, q50Aup, q84Aup, q97p5Aup = np.percentile(upA(params),
                                                          [2.5, 16, 50, 84, 97.5], axis=0)
q2p5gup, q16gup, q50gup, q84gup, q97p5gup = np.percentile(upg(params),
                                                          [2.5, 16, 50, 84, 97.5], axis=0)

q2p5Adown, q16Adown, q50Adown, q84Adown, q97p5Adown = np.percentile(downA(params),
                                                          [2.5, 16, 50, 84, 97.5], axis=0)
q2p5gdown, q16gdown, q50gdown, q84gdown, q97p5gdown = np.percentile(downg(params),
                                                          [2.5, 16, 50, 84, 97.5], axis=0)

/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aNpsr_down/45psr//chain_1.txt is loaded.

/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aNpsr_down/45psr//chain_1.txt is loaded.

/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aNpsr_up/45psr//chain_1.txt is loaded.

/data/taylor_group/william_lamb/GFL/middleton21/spsrs_10fCP_10firn_94/realisation_51/refits/m2aNpsr_up/45psr//chain_1.txt is loaded.



In [37]:
plt.rcParams["figure.figsize"] = [3.5503666805036667, 2.1942472810764047*2]
p = np.arange(1, 46)
fig, (ax1, ax2) = plt.subplots(nrows=2, tight_layout=True,
                               sharex=True)


ax1.plot(p, q50Aup, color='grey', label='Increasing $T_\mathrm{psr}$',
         alpha=0.75)
ax1.fill_between(p, q16Aup, q84Aup, alpha=0.1, color='grey')
ax1.fill_between(p, q2p5Aup, q97p5Aup, alpha=0.05, color='grey')
ax1.axhline(inj[0], c='k', ls='--');

ax2.plot(p, q50gup, color='grey', alpha=0.75)
ax2.fill_between(p, q16gup, q84gup, alpha=0.1, color='grey')
ax2.fill_between(p, q2p5gup, q97p5gup, alpha=0.05, color='grey')
ax2.axhline(inj[1], c='k', ls='--')

ax1.plot(p, q50Adown, label='Decreasing $T_\mathrm{psr}$',
         c='C0', alpha=1)
ax1.fill_between(p, q16Adown, q84Adown, alpha=0.2, color='C0')
ax1.fill_between(p, q2p5Adown, q97p5Adown, alpha=0.1, color='C0')
ax1.axhline(inj[0], c='k', ls='--');

ax2.plot(p, q50gdown, c='C0')
ax2.fill_between(p, q16gdown, q84gdown, alpha=0.2, color='C0')
ax2.fill_between(p, q2p5gdown, q97p5gdown, alpha=0.1, color='C0')
ax2.axhline(inj[1], c='k', ls='--')

ax2.set_xlabel('Number of pulsars')
ax1.set_ylabel('$\log_{10}A$')
ax2.set_ylabel('spectral index $\gamma$')

ax1.set_xlim(1,45)
ax1.legend(loc='lower right')

plt.show();
fig.savefig('../plots/per_psr.pdf',
            dpi=400, bbox_inches='tight');

# Let's do the 12.5yr data set!

In [9]:
# load paths to files of chains from FREE SPECTRUM REFIT
log10A_fsd, gamma_fsd = np.zeros((30, 145000)), np.zeros((30, 145000))
params = ['gwlog10_A', 'gwgamma']
for ii in tqdm(range(1, 31)):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/freespec/refits/m2aperfreq_v2//{ii}f/chain.core')
    log10A_fsd[ii-1] = c0('gwlog10_A')[-145000:]
    gamma_fsd[ii-1] = c0('gwgamma')[-145000:]

# calculate credible regions of the posterior
q2p5Afs, q16Afs, q50Afs, q84Afs, q97p5Afs = np.percentile(log10A_fsd,
                                                          [2.5, 16, 50, 84, 97.5],
                                                          axis=1)
q2p5gfs, q16gfs, q50gfs, q84gfs, q97p5gfs = np.percentile(gamma_fsd,
                                                          [2.5, 16, 50, 84, 97.5],
                                                          axis=1)

  3%|▎         | 1/30 [00:00<00:05,  5.60it/s]

  7%|▋         | 2/30 [00:00<00:07,  3.75it/s]

 13%|█▎        | 4/30 [00:01<00:12,  2.15it/s]

 17%|█▋        | 5/30 [00:02<00:11,  2.22it/s]

 23%|██▎       | 7/30 [00:02<00:06,  3.80it/s]

...

 83%|████████▎ | 25/30 [00:07<00:01,  3.96it/s]

 90%|█████████ | 27/30 [00:08<00:01,  2.29it/s]

 93%|█████████▎| 28/30 [00:09<00:01,  1.91it/s]

 97%|█████████▋| 29/30 [00:10<00:00,  1.98it/s]

100%|██████████| 30/30 [00:11<00:00,  2.53it/s]


In [10]:
# load paths to files of chains from FREE SPECTRUM REFIT
log10A_fsu, gamma_fsu = np.zeros((30, 145000)), np.zeros((30, 145000))
params = ['gwlog10_A', 'gwgamma']
for ii in tqdm(range(1, 31)):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/freespec/refits/m2aperfreq_rev_v2/{ii}f/chain.core')
    log10A_fsu[ii-1] = c0('gwlog10_A')[-145000:]
    gamma_fsu[ii-1] = c0('gwgamma')[-145000:]

# calculate credible regions of the posterior
q2p5Afsd, q16Afsd, q50Afsd, q84Afsd, q97p5Afsd = np.percentile(log10A_fsu,
                                                               [2.5, 16, 50, 84, 97.5],
                                                               axis=1)
q2p5gfsd, q16gfsd, q50gfsd, q84gfsd, q97p5gfsd = np.percentile(gamma_fsu,
                                                               [2.5, 16, 50, 84, 97.5],
                                                               axis=1)

  0%|          | 0/30 [00:00<?, ?it/s]

  3%|▎         | 1/30 [00:01<00:55,  1.92s/it]

 10%|█         | 3/30 [00:06<00:50,  1.88s/it]

 17%|█▋        | 5/30 [00:07<00:25,  1.00s/it]

 20%|██        | 6/30 [00:07<00:18,  1.30it/s]

...

 77%|███████▋  | 23/30 [00:20<00:03,  2.00it/s]

 87%|████████▋ | 26/30 [00:20<00:01,  3.96it/s]

 90%|█████████ | 27/30 [00:22<00:02,  1.35it/s]

 93%|█████████▎| 28/30 [00:23<00:01,  1.57it/s]

100%|██████████| 30/30 [00:24<00:00,  1.24it/s]

In [28]:
log10As, gammas = np.zeros((5, 145000)), np.zeros((5, 145000))
params = ['gwlog10_A', 'gwgamma']
for ii in tqdm(range(1, 6)):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/spsrs/5fFScp_30fPLirn94/refits/m2aperfreq/{ii}f/chain.core')
    log10As[ii-1] = c0('gwlog10_A')[-145000:]
    gammas[ii-1] = c0('gwgamma')[-145000:]
    
# calculate credible regions of the posterior
q2p5A, q16A, q50A, q84A, q97p5A = np.percentile(log10As,
                                                [2.5, 16, 50, 84, 97.5], axis=1,)
q2p5g, q16g, q50g, q84g, q97p5g = np.percentile(gammas,
                                                [2.5, 16, 50, 84, 97.5], axis=1,)

 20%|██        | 1/5 [00:01<00:04,  1.10s/it]

 60%|██████    | 3/5 [00:04<00:03,  1.54s/it]

100%|██████████| 5/5 [00:08<00:00,  1.73s/it]

In [25]:
log10A_down, gamma_down = np.zeros((5, 145000)), np.zeros((5, 145000))
params = ['gwlog10_A', 'gwgamma']
for ii in tqdm(range(1, 6)):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/spsrs/5fFScp_30fPLirn94/refits/m2aperfreq_rev/{ii}f/chain.core')
    log10A_down[ii-1] = c0('gwlog10_A')[-145000:]
    gamma_down[ii-1] = c0('gwgamma')[-145000:]

# calculate credible regions of the posterior
q2p5Ad, q16Ad, q50Ad, q84Ad, q97p5Ad = np.percentile(log10A_down,
                                                     [2.5, 16, 50, 84, 97.5],
                                                     axis=1)
q2p5gd, q16gd, q50gd, q84gd, q97p5gd = np.percentile(gamma_down,
                                                     [2.5, 16, 50, 84, 97.5],
                                                     axis=1)

  0%|          | 0/5 [00:00<?, ?it/s]

 40%|████      | 2/5 [00:01<00:01,  2.28it/s]

 80%|████████  | 4/5 [00:02<00:00,  1.96it/s]

100%|██████████| 5/5 [00:03<00:00,  1.58it/s]


In [19]:
downA_125, downg_125 = np.zeros((45, 145000)), np.zeros((45, 145000))
params = ['gwlog10_A', 'gwgamma']
for ii in tqdm(range(1, 46)):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/spsrs/5fFScp_30fPLirn94/refits/m2aNpsr_down/{ii}psr/chain.core')
    downA_125[ii-1] = c0('gwlog10_A')[-145000:]
    downg_125[ii-1] = c0('gwgamma')[-145000:]
    
q2p5Adown, q16Adown, q50Adown, q84Adown, q97p5Adown = np.percentile(downA_125,
                                                                    [2.5, 16, 50, 84, 97.5], axis=1)
q2p5gdown, q16gdown, q50gdown, q84gdown, q97p5gdown = np.percentile(downg_125,
                                                                    [2.5, 16, 50, 84, 97.5], axis=1)

  0%|          | 0/45 [00:00<?, ?it/s]

  2%|▏         | 1/45 [00:00<00:43,  1.02it/s]

  4%|▍         | 2/45 [00:01<00:36,  1.16it/s]

  7%|▋         | 3/45 [00:02<00:31,  1.33it/s]

 11%|█         | 5/45 [00:03<00:18,  2.12it/s]

...

 87%|████████▋ | 39/45 [00:21<00:00,  6.19it/s]

 89%|████████▉ | 40/45 [00:21<00:00,  6.70it/s]

 91%|█████████ | 41/45 [00:25<00:04,  1.20s/it]

 96%|█████████▌| 43/45 [00:25<00:01,  1.31it/s]

100%|██████████| 45/45 [00:27<00:00,  1.66it/s]


In [31]:
upA_125, upg_125 = np.zeros((45, 145000)), np.zeros((45, 145000))
params = ['gwlog10_A', 'gwgamma']
for ii in tqdm(range(1, 46)):
    c0 = co.Core(corepath=f'/data/taylor_group/william_lamb/analyses/NANOGrav/NG12p5/spsrs/5fFScp_30fPLirn94/refits/m2aNpsr_up_best/{ii}psr/chain.core')
    upA_125[ii-1] = c0('gwlog10_A')[-145000:]
    upg_125[ii-1] = c0('gwgamma')[-145000:]
    
q2p5Aup, q16Aup, q50Aup, q84Aup, q97p5Aup = np.percentile(upA_125,
                                                          [2.5, 16, 50, 84, 97.5], axis=1)
q2p5gup, q16gup, q50gup, q84gup, q97p5gup = np.percentile(upg_125,
                                                          [2.5, 16, 50, 84, 97.5], axis=1)

  0%|          | 0/45 [00:00<?, ?it/s]

  4%|▍         | 2/45 [00:02<00:48,  1.14s/it]

  7%|▋         | 3/45 [00:02<00:28,  1.48it/s]

  9%|▉         | 4/45 [00:03<00:21,  1.92it/s]

 11%|█         | 5/45 [00:03<00:24,  1.63it/s]

...

 87%|████████▋ | 39/45 [00:41<00:04,  1.43it/s]

 89%|████████▉ | 40/45 [00:42<00:03,  1.51it/s]

 93%|█████████▎| 42/45 [00:44<00:02,  1.29it/s]

 98%|█████████▊| 44/45 [00:46<00:00,  1.28it/s]

100%|██████████| 45/45 [00:46<00:00,  1.03s/it]


In [32]:
plt.rcParams["figure.figsize"] = [3.5503666805036667*2, 2.1942472810764047*2]

In [33]:
Tspan2 = 407576851.48121357
freqs2 = np.arange(1, 31)/Tspan2

In [34]:
fig, ((ax1, ax2),
      (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, tight_layout=True,
                                 sharey='row', sharex='col')

f = np.arange(1,31)
p = np.arange(1,46)

#ax1.set_title('Free spectrum refit')
#ax2.set_title('GFL Lite')

ax1.plot(f, q50Afs, c='C0', label='FSR - Increasing frequency')
ax1.fill_between(f, q16Afs, q84Afs, alpha=0.2, color='C0')
ax1.fill_between(f, q2p5Afs, q97p5Afs, alpha=0.1, color='C0')
ax1.axhline(inj[0], c='k', ls='--', alpha=0.75);

ax3.plot(f, q50gfs, c='C0')
ax3.fill_between(f, q16gfs, q84gfs, alpha=0.2, color='C0')
ax3.fill_between(f, q2p5gfs, q97p5gfs, alpha=0.1, color='C0')
ax3.axhline(inj[1], c='k', ls='--', alpha=0.75)

ax1.plot(f, q50Afsd, c='grey', alpha=0.75,
         label='FSR - Decreasing frequency')
ax1.fill_between(f, q16Afsd, q84Afsd, alpha=0.1, color='grey')
ax1.fill_between(f, q2p5Afsd, q97p5Afsd, alpha=0.1, color='grey')
ax1.axhline(inj[0], c='k', ls='--', alpha=0.75);

ax3.plot(f, q50gfsd, c='grey', alpha=0.75)
ax3.fill_between(f, q16gfsd, q84gfsd, alpha=0.1, color='grey')
ax3.fill_between(f, q2p5gfsd, q97p5gfsd, alpha=0.1, color='grey')
ax3.axhline(inj[1], c='k', ls='--', alpha=0.75)

# GFL Lite freq refit # 

ax1.plot(f[:5], q50A, color='C2', ls='-',
         label='GFL Lite - Increasing frequency')
ax1.fill_between(f[:5], q16A, q84A, alpha=0.4, fc='none', ec='g', hatch='.....')
ax1.fill_between(f[:5], q2p5A, q97p5A, alpha=0.1, fc='g')
ax1.axhline(inj[0], c='k', ls='--', alpha=0.25);

ax3.plot(f[:5], q50g, color='C2', ls='-')
ax3.fill_between(f[:5], q16g, q84g, alpha=0.4, fc='none', ec='g', hatch='.....')
ax3.fill_between(f[:5], q2p5g, q97p5g, alpha=0.1, fc='g')
ax3.axhline(inj[1], c='k', ls='--', alpha=0.25)

"""ax1.plot(f[:5], q50Ad, color='grey', alpha=0.75, label='Decreasing frequency')
ax1.fill_between(f[:5], q16Ad, q84Ad, alpha=0.1, color='grey')
ax1.fill_between(f[:5], q2p5Ad, q97p5Ad, alpha=0.05, color='grey')
ax1.axhline(inj[0], c='k', ls='--');

ax3.plot(f[:5], q50gd, color='grey', alpha=0.75)
ax3.fill_between(f[:5], q16gd, q84gd, alpha=0.1, color='grey')
ax3.fill_between(f[:5], q2p5gd, q97p5gd, alpha=0.05, color='grey')
ax3.axhline(inj[1], c='k', ls='--')"""

ax1.set_xlim(1,31)
ax1.legend(loc='lower right');

# GFL Lite psr refit #

ax2.plot(p, q50Adown, color='C2', ls='--',
         label='GFL Lite - Decreasing $T_\mathrm{psr}$',)
ax2.fill_between(p, q16Adown, q84Adown, alpha=0.2, color='C2')
ax2.fill_between(p, q2p5Adown, q97p5Adown, alpha=0.1, color='C2')
ax2.axhline(inj[0], c='k', ls='--', alpha=0.75);

ax4.plot(p, q50gdown, color='C2', ls='--')
ax4.fill_between(p, q16gdown, q84gdown, alpha=0.2, color='C2')
ax4.fill_between(p, q2p5gdown, q97p5gdown, alpha=0.1, color='C2')
ax4.axhline(inj[1], c='k', ls='--', alpha=0.75)

ax2.plot(p, q50Aup, color='grey', alpha=0.75,
         label='GFL Lite - Increasing $T_\mathrm{psr}$')
ax2.fill_between(p, q16Aup, q84Aup, alpha=0.1, color='grey')
ax2.fill_between(p, q2p5Aup, q97p5Aup, alpha=0.05, color='grey')
ax2.axhline(inj[0], c='k', ls='--', alpha=0.75);

ax4.plot(p, q50gup, color='grey', alpha=0.75)
ax4.fill_between(p, q16gup, q84gup, alpha=0.1, color='grey')
ax4.fill_between(p, q2p5gup, q97p5gup, alpha=0.05, color='grey')
ax4.axhline(inj[1], c='k', ls='--', alpha=0.75)

ax3.set_xlabel('Number of frequencies')
ax4.set_xlabel('Number of pulsars')
ax1.set_ylabel('$\log_{10}A$')
ax3.set_ylabel('spectral index $\gamma$')

ax2.set_xlim(1,45)
ax2.legend(loc='lower right')

secx = ax1.secondary_xaxis('top')
secx.set_xticks(np.array([1, 5, 10, 15, 20, 25, 30]))
labels1 = secx.set_xticklabels([f'{freq:.1f}' for freq in freqs2[[0, 4, 9, 14, 19, 24, 29]]*10**9])
secx.set_xlabel('GW frequencies [nHz]')

[lab.set_color("C0") for lab in labels1]

ax3.set_xlim(1,30)

fig.savefig('../plots/per_freq_12p5.pdf',
            dpi=400, bbox_inches='tight');

plt.show();

In [35]:
plt.close();